# PPT_ACHR_1

In [ ]:
%matplotlib widget
%reload_ext autoreload
%autoreload 2

import numpy as np
import scipy
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
import yaml
import re

import hypnogram as hp
from ecephys.utils import replace_outliers
import ecephys.plot as eplt
import ecephys.signal.timefrequency as tfr
import ecephys.signal.kd_utils as kd
import ecephys.signal.kd_plotting as kp
import ecephys.signal.kd_pax as kpx
import tdt_xarray as tx

bands_def = dict(delta=(0.5, 4.1), theta=(4.1, 8), beta = (13, 20), low_gamma = (40, 55), high_gamma = (65, 80))

kd_ref = {}
kd_ref['echans'] = [1,2,3]
kd_ref['fchans']=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
kd_ref['analysis_root'] = Path(r"L:\Data\ACHR_PROJECT_MATERIALS\PPT_ACHR_1\analysis_data")
kd_ref['tank_root'] = Path(r"L:\Data\PPT_ACHR_1_TANK")

# Exp-1

In [ ]:
data = tdt.read_block(e1bl1p, t1=0, t2=10)

In [ ]:
data.streams['LFPr'].channels

In [ ]:
channels=[12, 13, 11, 14, 10, 15, 9, 16, 5, 4, 6, 3, 7, 2, 8, 1]

In [ ]:
channels

In [ ]:
data.epocs.Bn1_

In [ ]:
e1p = "L:\Data\PPT_ACHR_1_TANK\PPT_ACHR_1-exp1"
e1bl1p = "L:\Data\PPT_ACHR_1_TANK\PPT_ACHR_1-exp1-bl1"
e1bl2p = "L:\Data\PPT_ACHR_1_TANK\PPT_ACHR_1-exp1-bl2"
hyp_path = "L:\Data\ACHR_PROJECT_MATERIALS\PPT_ACHR_1\laser_stims_8_27.txt"

In [ ]:
e1_hyp = hp.load_visbrain_hypnogram(hyp_path)

In [ ]:
x1be, x1be_spg, x1be_bp = get_dsb(e1bl1p, bands, t1=0, t2=0, channel=[1,2,3], store='EEGr')
x1be2, x1be2_spg, x1be2_bp = get_dsb(e1bl2p, bands, t1=0, t2=0, channel=[1,2,3], store='EEGr')
x1e, x1e_spg, x1e_bp = get_dsb(e1p, bands, t1=0, t2=0, channel=[1,2,3], store='EEGr')

In [ ]:
x1bf, x1bf_spg, x1bf_bp = get_dsb(e1bl1p, bands, t1=0, t2=0, channel=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], store='LFPr')
x1bf2, x1bf2_spg, x1bf2_bp = get_dsb(e1bl2p, bands, t1=0, t2=0, channel=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], store='LFPr')

In [ ]:
x1f, x1f_spg, x1f_bp = get_dsb(e1p, bands, t1=0, t2=0, channel=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], store='LFP_')

In [ ]:
x1bf.to_netcdf(r"L:\Data\ACHR_PROJECT_MATERIALS\PPT_ACHR_1\analysis_data\x1bf.nc")
x1bf_spg.to_netcdf(r"L:\Data\ACHR_PROJECT_MATERIALS\PPT_ACHR_1\analysis_data\x1bf_spg.nc")
x1bf_bp.to_netcdf(r"L:\Data\ACHR_PROJECT_MATERIALS\PPT_ACHR_1\analysis_data\x1bf_bp.nc")

In [ ]:
x1bf2.to_netcdf(r"L:\Data\ACHR_PROJECT_MATERIALS\PPT_ACHR_1\analysis_data\x1bf2.nc")
x1bf2_spg.to_netcdf(r"L:\Data\ACHR_PROJECT_MATERIALS\PPT_ACHR_1\analysis_data\x1bf2_spg.nc")
x1bf2_bp.to_netcdf(r"L:\Data\ACHR_PROJECT_MATERIALS\PPT_ACHR_1\analysis_data\x1bf2_bp.nc")

## Plots

### EEG Plots

In [ ]:
channels=[1,2,3]
for chan in channels:
    b, s = kp.plot_spectrogram_with_bandpower(x1e_spg, x1e_bp.delta, hyp=e1_hyp, channel=chan, start_time=0, end_time=x1e_spg.time.values.max(), title="Delta Power, EEG-"+str(chan)+", Exp-1")
    b.axvspan(
            42294,
            85494,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )
    b.set_ylim(0, 4000)

In [ ]:
channels=[1,2,3]
for chan in channels:
    b, s = kp.plot_spectrogram_with_bandpower(x1be_spg, x1be_bp.delta, hyp=None, channel=chan, start_time=0, end_time=x1be_spg.time.values.max(), title="Delta Power, EEG-"+str(chan)+", Exp-1 Baseline (9am-6pm)")
    b.set_ylim(0, 4000)

In [ ]:
channels=[1,2,3]
for chan in channels:
    b, s = kp.plot_spectrogram_with_bandpower(x1be2_spg, x1be2_bp.delta, hyp=None, channel=chan, start_time=0, end_time=x1be2_spg.time.values.max(), title="Delta Power, EEG-"+str(chan)+", Exp-1 Baseline Part-2 (6pm-9am)")
    b.axvspan(
            9524,
            52724,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )
    b.set_ylim(0, 4000)

### LFP Plots

In [ ]:
channels=[1,9,13]
for chan in channels:
    b, s = kp.plot_spectrogram_with_bandpower(x1f_spg, x1f_bp.delta, hyp=e1_hyp, channel=chan, start_time=0, end_time=x1f_spg.time.max(), title="Delta Power, LFP-"+str(chan)+", Exp-1")
    b.axvspan(
            42294,
            85494,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )
    b.set_ylim(0, 130000)



In [ ]:
band = 'delta'
channels=[1,9,13]
for chan in channels:
    b, s = kp.plot_spectrogram_with_bandpower(x1bf_spg, x1bf_bp, band=band, hyp=None, channel=chan, start_time=0, end_time=20000, title=band.capitalize()+" Power, LFP-"+str(chan)+", Exp-1 Baseline (9am-6pm)")
    b.set_ylim(0, 130000)

In [ ]:
band = 'delta'
channels=[1,9,13]
for chan in channels:
    b, s = kp.plot_spectrogram_with_bandpower(x1bf2_spg, x1bf2_bp, band=band, hyp=None, channel=chan, start_time=0, end_time=x1bf2_spg.time.max(), title=band.capitalize()+" Power, LFP-"+str(chan)+", Exp-1 Baseline Part-2 (6pm-9am)")
    b.axvspan(
            9524,
            52724,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )
    b.set_ylim(0, 130000)

# Exp-2

In [ ]:
e2p = "L:\Data\PPT_ACHR_1_TANK\PPT_ACHR_1-exp2"
e2blp = "L:\Data\PPT_ACHR_1_TANK\PPT_ACHR_1-exp2-bl"

In [ ]:
p1p = "L:\Data\PPT_ACHR_1_TANK\PPT_ACHR_1-exp2-post1"
p2p = "L:\Data\PPT_ACHR_1_TANK\PPT_ACHR_1-exp2-post2"

In [ ]:
tp = "L:\Data\PPT_ACHR_1_TANK\PPT_ACHR_1-test_sept3" 


In [ ]:
t1f, t1f_spg, t1f_bp = get_dsb(tp, bands, t1=0, t2=0, channel=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], store='LFP_')
t1e, t1e_spg, t1e_bp = get_dsb(tp, bands, t1=0, t2=0, channel=[1,2,3], store='EEG_')

In [ ]:
x2be, x2be_spg, x2be_bp = get_dsb(e2blp, bands, t1=0, t2=0, channel=[1,2,3], store='EEG_')
x2e, x2e_spg, x2e_bp = get_dsb(e2p, bands, t1=0, t2=0, channel=[1,2,3], store='EEG_')

In [ ]:
x2bf, x2bf_spg, x2bf_bp = get_dsb(e2blp, bands, t1=0, t2=0, channel=[1,9,13], store='LFP_')

x2f, x2f_spg, x2f_bp = get_dsb(e2p, bands, t1=0, t2=0, channel=[1,9,13], store='LFP_')

In [ ]:
p2f, p2f_spg, p2f_bp = get_dsb(p2p, bands, t1=0, t2=0, channel=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], store='LFP_')
p2e, p2e_spg, p2e_bp = get_dsb(p2p, bands, t1=0, t2=0, channel=[1,2,3], store='EEG_')

## Plots

### EEG Plots

In [ ]:
channels=[1,2,3]
for chan in channels:
    b, s = kp.plot_spectrogram_with_bandpower(x2be_spg, x2be_bp.delta, hyp=None, channel=chan, start_time=0, end_time=x2be_spg.time.values.max(), title="Delta Power, EEG-"+str(chan)+", Exp-2 Baseline (9am-9am)")
    b.set_ylim(0, 4000)
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )

In [ ]:
channels=[1,2,3]
for chan in channels:
    b, s = kp.plot_spectrogram_with_bandpower(x2e_spg, x2e_bp.delta, hyp=None, channel=chan, start_time=0, end_time=x2e_spg.time.values.max(), title="Delta Power, EEG-"+str(chan)+", Exp-2 (9am-9am)")
    b.axvspan(
            55.69,
            21614.44,
            alpha=0.3,
            color='royalblue',
            zorder=1000,
            ec="none",
        )
    b.set_ylim(0, 4000)
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )

### LFP Plots

In [ ]:
channels=[1,9,13]
for chan in channels:
    b, s = kp.plot_spectrogram_with_bandpower(x2bf_spg, x2bf_bp.high_gamma, hyp=None, channel=chan, start_time=0, end_time=x2bf_spg.time.max(), title="High Gamma (65-80Hz) Power, LFP-"+str(chan)+", Exp-2 Baseline (9am-9am)")
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )
    b.set_ylim(0, 1600)

In [ ]:
channels=[1,9,13]
for chan in channels:
    b, s = kp.plot_spectrogram_with_bandpower(x2f_spg, x2f_bp.high_gamma, hyp=None, channel=chan, start_time=0, end_time=x2f_spg.time.max(), title="High Gamma (65-80Hz) Power, LFP-"+str(chan)+", Exp-2 (9am-9am)")
    b.axvspan(
            0,
            24600,
            alpha=0.3,
            color='royalblue',
            zorder=1000,
            ec="none",
        )
    b.set_ylim(0, 1600)
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )

In [ ]:
channels=[1,9,13]
for chan in channels:
    b, s = kp.plot_spectrogram_with_bandpower(x2bf_spg, x2bf_bp.low_gamma, hyp=None, channel=chan, start_time=0, end_time=x2bf_spg.time.max(), title="Low Gamma (40-55Hz) Power, LFP-"+str(chan)+", Exp-2 Baseline (9am-9am)")
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )
    b.set_ylim(0, 5000)

In [ ]:
channels=[1,9,13]
for chan in channels:
    b, s = kp.plot_spectrogram_with_bandpower(x2f_spg, x2f_bp.low_gamma, hyp=None, channel=chan, start_time=0, end_time=x2f_spg.time.max(), title="Low Gamma Power, LFP-"+str(chan)+", Exp-2 (9am-9am)")
    b.axvspan(
            0,
            24600,
            alpha=0.3,
            color='royalblue',
            zorder=1000,
            ec="none",
        )
    b.set_ylim(0, 5000)
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )

### Post-Plots

In [ ]:
channels=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
for chan in channels:
    b, s = kp.plot_spectrogram_with_bandpower(p2f_spg, p2f_bp.delta, hyp=None, channel=chan, start_time=0, end_time=p2f_spg.time.max(), title="Delta Power, LFP-"+str(chan)+", Exp-2-POST")
    b.set_ylim(0, 200000)


In [ ]:
channels=[1,2,3]
for chan in channels:
    b, s = kp.plot_spectrogram_with_bandpower(p2e_spg, p2e_bp.delta, hyp=None, channel=chan, start_time=0, end_time=p2e_spg.time.values.max(), title="Delta Power, EEG-"+str(chan)+", Exp-2-POST")
    b.set_ylim(0, 4000)
    

## Scratch

In [ ]:
channels=[1,2,3]
for chan in channels:
    b, s = kp.plot_spectrogram_with_bandpower(t1e_spg, t1e_bp.delta, hyp=None, channel=chan, start_time=0, end_time=t1e_spg.time.values.max(), title="Delta Power, EEG-"+str(chan)+", Sept. 3 Night")
    b.set_ylim(0, 4000)

In [ ]:
channels=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
for chan in channels:
    b, s = kp.plot_spectrogram_with_bandpower(t1f_spg, t1f_bp.delta, hyp=None, channel=chan, start_time=0, end_time=t1f_spg.time.values.max(), title="Delta Power, LFP-"+str(chan)+", Sept. 3 Night")
    #b.set_ylim(0, 4000)

# Exp-3

In [ ]:
x3p = "L:\Data\PPT_ACHR_1_TANK\PPT_ACHR_1-exp3"
x3blp = "L:\Data\PPT_ACHR_1_TANK\PPT_ACHR_1-exp3-bl"


In [ ]:
epocs = tdt.read_block(x3p, evtype=['epocs'])

In [ ]:
epocs.epocs.Bn1_

In [ ]:
x3_stims['2.1.on'] = 26040
x3_stims['2.1.off'] = 26941
x3_stims['2.2.on'] = 27242
x3_stims['2.2.off'] = 28144
x3_stims['2.3.on'] = 28497
x3_stims['2.3.off'] = 29412

In [ ]:
"Use get_data_spg to create xset"

In [ ]:
x3_key_list = ['e', 'be', 'f', 'bf', 'e_spg', 'be_spg', 'f_spg', 'bf_spg']

In [ ]:
save_xset(x3, analysis_root=ar)

In [ ]:
x3 = kd.fetch_xset('x3', x3_key_list, ar)

In [ ]:
bp = kd.get_bp_set(x3['e_spg'], bands)['delta']

## Plots

In [ ]:
t = x3["be_spg"]

In [ ]:
band='delta'
channels=[1,2,3]
for chan in channels:
    b, s = kp.plot_spectrogram_with_bandpower(x3['be_spg'], bands_def, band, hyp=None, channel=chan, start_time=0, end_time=x3['be_spg'].time.max(), title=band.capitalize()+" Power, EEG-"+str(chan)+", Exp-3 Baseline (9am-9am)")
    b.set_ylim(0, 4000)
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )

In [ ]:
band='delta'
for chan in echans:
    b, s = kp.plot_spectrogram_with_bandpower(x3['e_spg'], bands_def, band, hyp=None, channel=chan, start_time=0, end_time=x3['e_spg'].time.max(), title=band.capitalize()+" Power, EEG-"+str(chan)+", Exp-3 (9am-10am)")
    b.set_ylim(0, 4000)
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )
    b.axvspan(
            x3_stims["s1.start"],
            x3_stims["s1.stop"],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )
    b.axvspan(
            x3_stims["s2.start"],
            x3_stims["s2.stop"],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )
    b.axvspan(
            x3_stims["s3.start"],
            x3_stims["s3.stop"],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )

### EEG-plots

In [ ]:
band='delta'
chans = [1,2,3]
for chan in chans:
    b, s = kp.plot_spectrogram_with_bandpower(x3['e_spg'], bands_def, band, hyp=None, channel=chan, start_time=30100, end_time=32000, title=band.capitalize()+" Power, EEG-"+str(chan)+", Exp-3, Stimulation #3")
    b.set_ylim(0, 4000)
    for on in ons:
        b.axvline(x=on, alpha=0.05, color='lightskyblue')
    

In [ ]:
band='delta'
chans = [3,9,16]
for chan in echans:
    b, s = kp.plot_spectrogram_with_bandpower(x3['e_spg'], bands_def, band, hyp=None, channel=chan, start_time=30000, end_time=40000, title=band.capitalize()+" Power, EEG-"+str(chan)+", Exp-3, Stimulation #4")
    b.set_ylim(0, 4000)
    b.axvspan(
            x3_stims["s4.start"],
            x3_stims["s4.stop"],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )

In [ ]:
band='delta'
chans = [3,9,16]
for chan in echans:
    b, s = kp.plot_spectrogram_with_bandpower(x3['e_spg'], bands_def, band, hyp=None, channel=chan, start_time=23000, end_time=33000, title=band.capitalize()+" Power, EEG-"+str(chan)+", Exp-3, Stimulation #2")
    b.set_ylim(0, 4000)
    b.axvspan(
            x3_stims["2.1.on"],
            x3_stims["2.1.off"],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )
    b.axvspan(
            x3_stims["2.2.on"],
            x3_stims["2.2.off"],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )
    b.axvspan(
            x3_stims["2.3.on"],
            x3_stims["2.3.off"],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )

### LFP-Plots

In [ ]:
band='delta'
chans = [3,9,16]
for chan in chans:
    b, s = kp.plot_spectrogram_with_bandpower(x3['bf_spg'], bands_def, band, hyp=None, channel=chan, start_time=0, end_time=x3['bf_spg'].time.max(), title=band.capitalize()+" Power, LFP-"+str(chan)+", Exp-3 Baseline (9am-9am)")
    b.set_ylim(0, 130000)
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )

In [ ]:
band='delta'
chans = [3,9,16]
for chan in chans:
    b, s = kp.plot_spectrogram_with_bandpower(x3['f_spg'], bands_def, band, hyp=None, channel=chan, start_time=30100, end_time=32000, title=band.capitalize()+" Power, LFP-"+str(chan)+", Exp-3, Stimulation #3")
    b.set_ylim(0, 130000)
    for on in ons:
        b.axvline(x=on, alpha=0.05, color='lightskyblue')
    

In [ ]:
band='delta'
chans = [3,9,16]
for chan in chans:
    b, s = kp.plot_spectrogram_with_bandpower(x3['f_spg'], bands_def, band, hyp=None, channel=chan, start_time=0, end_time=x3['f_spg'].time.max(), title=band.capitalize()+" Power, LFP-"+str(chan)+", Exp-3 (9am-10am)")
    b.set_ylim(0, 130000)
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )
    b.axvspan(
            x3_stims["s1.start"],
            x3_stims["s1.stop"],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )
    b.axvspan(
            x3_stims["s2.start"],
            x3_stims["s2.stop"],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )
    b.axvspan(
            x3_stims["s3.start"],
            x3_stims["s3.stop"],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )

In [ ]:
x3_stims["s2.stop"]

In [ ]:
band='delta'
chans = [3,9,16]
for chan in chans:
    b, s = kp.plot_spectrogram_with_bandpower(x3['f_spg'], bands_def, band, hyp=None, channel=chan, start_time=23000, end_time=33000, title=band.capitalize()+" Power, LFP-"+str(chan)+", Exp-3, Stimulation #2")
    b.set_ylim(0, 130000)
    b.axvspan(
            x3_stims["2.1.on"],
            x3_stims["2.1.off"],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )
    b.axvspan(
            x3_stims["2.2.on"],
            x3_stims["2.2.off"],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )
    b.axvspan(
            x3_stims["2.3.on"],
            x3_stims["2.3.off"],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )

In [ ]:
band='delta'
chans = [3,9,16]
for chan in chans:
    b, s = kp.plot_spectrogram_with_bandpower(x3['f_spg'], bands_def, band, hyp=None, channel=chan, start_time=30000, end_time=40000, title=band.capitalize()+" Power, LFP-"+str(chan)+", Exp-3, Stimulation #4")
    b.set_ylim(0, 130000)
    b.axvspan(
            x3_stims["s4.start"],
            x3_stims["s4.stop"],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )

# Exp-4

In [ ]:
e4p = "L:\Data\PPT_ACHR_1_TANK\PPT_ACHR_1-exp4"

In [ ]:
a1x4 = {}
a1x4['e'], a1x4['e_spg'] = kd.get_data_spg(e4p, store='EEGr', t1=0, t2=0, channel=kd_ref['echans'])
a1x4['f'], a1x4['f_spg'] = kd.get_data_spg(e4p, store='LFP_', t1=0, t2=0, channel=kd_ref['fchans'])

In [ ]:
a1x4['epocs'] = tdt.read_block(e4p, evtype=['epocs'])

In [ ]:
a1x4['onset'] = 2487.19306752
a1x4['offset'] = 16887.86915328

In [ ]:
a1x4['name'] = 'a1x4'

kd.save_xset(a1x4, kd_ref['analysis_root'])

In [ ]:
kl4 = kd.get_key_list(a1x4)
kd.save_key_list(kl4)

In [ ]:
a1x4['e'].datetime.max()

In [ ]:
band='delta'
for chan in kd_ref['echans']:
    b, s = kp.plot_spectrogram_with_bandpower(a1x4['e_spg'], bands_def, band, hyp=None, channel=chan, start_time=0, end_time=a1x4['e_spg'].time.max(), title=band.capitalize()+" Power, EEG-"+str(chan)+", Exp-4 (4-Hr Stim)")
    b.set_ylim(0, 4000)
    b.axvspan(
            a1x4['onset'],
            a1x4['offset'],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )
    b.axvspan(
            129600,
            172800,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )

In [ ]:
plt.close('all')
band='delta'
for chan in kd_ref['echans']:
    b, s = kp.plot_spectrogram_with_bandpower(a1x4['e_spg'], bands_def, band, hyp=None, channel=chan, start_time=0, end_time=43200, title=band.capitalize()+" Power, EEG-"+str(chan)+", Exp-4 (4-Hr Stim)")
    b.set_ylim(0, 4000)
    b.axvspan(
            a1x4['onset'],
            a1x4['offset'],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )

In [ ]:
plt.close('all')
band='delta'
for chan in [3,11,16]:
    b, s = kp.plot_spectrogram_with_bandpower(a1x4['f_spg'], bands_def, band, hyp=None, channel=chan, start_time=0, end_time=43200, title=band.capitalize()+" Power, LFP-"+str(chan)+", Exp-4 (4-Hr Stim)")
    b.set_ylim(0, 120000)
    b.axvspan(
            a1x4['onset'],
            a1x4['offset'],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )